11 ЛАБ. РАБОТА

In [91]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer # инструмент ипользуется для преобразования текстовых данных в матрицу счетчиков токенов. 
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split

Чтение файла. Датасет "Emotion Detection from text"

In [92]:
df = pd.read_csv("tweet_emotions.csv", sep=",")
df.dropna()
df

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
...,...,...,...
39995,1753918954,neutral,@JohnLloydTaylor
39996,1753919001,love,Happy Mothers Day All my love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


Деление данных на тестовую и обучающие выборки

In [93]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['tweet_id', 'sentiment'], axis = 1), df['sentiment'], test_size=0.2, random_state=42)
X_train


,content
14307,Thundershowers plus baseball equals awwww
17812,The Hangover: That movie is gonna be hilarious...
11020,Playing some games before I leave for work W...
15158,About to be stick at work till freakin 2 am. ...
24990,@Sarah_x_ATL u mean jack barakat's?! wow so ha...
...,...
6265,Cleaning
11284,ooo my goddddd; vodafone trouble. no network
38158,watching run fat boy run...haha its soo funny.
860,@calvinharris make the most of pork products w...


Bag of Words (мешок слов)


Позволяет представить текст в виде вектора, где каждый элемент вектора соответствует определенному слову, а значение элемента - количество раз, которое это слово встречается в тексте.

In [94]:
## Объединение обучающего и тестового DataFrame
combined_data = pd.concat([X_train, X_test])

vectorizer = CountVectorizer() #экземпляр

#обучаем модель
vectorizer.fit(combined_data.content)

CountVectorizer()

Преобразование текста в матрицу счётчиков

In [95]:
#словари
X_train = vectorizer.transform(X_train.content)
X_test = vectorizer.transform(X_test.content)
y_train = y_train.values
y_test = y_test.values

Обучение классификатора

In [96]:
classifier = LogisticRegression() #лог. регрессия
from sklearn.tree import DecisionTreeClassifier
#classifier = DecisionTreeClassifier()

classifier.fit(X_train,y_train)

C:\Users\79526\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Предсказание и расчёт метрик

In [97]:
y_pred = classifier.predict(X_test)

pres = precision_score(y_test,y_pred,average='weighted') #точность
print(f"Precision: {pres}")

recall = recall_score(y_test,y_pred,average='weighted') #отклик
print(f"Recall: {recall}")

f1 = f1_score(y_test,y_pred,average='weighted') #сбалансированным показателем между точностью и полнотой (гармоническое среднее)
print(f"F1:: {f1}")


Precision: 0.30493297908610556
Recall: 0.333625
F1:: 0.309641983606121


C:\Users\79526\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Предобработка (для повышения показателей)

In [98]:
from nltk.stem import WordNetLemmatizer

# Создаем объект лемматизатора
lemat = WordNetLemmatizer()

def lematize_word(text):
    #Разделяем текст на слова и применяем лемматизатор к каждому слову и объединяем обратно в текст
    return " ".join([lemat.lemmatize(word) for word in text.split()])

In [99]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#убираем английские стоп-слова
eng_stopwords = set(stopwords.words("english"))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in eng_stopwords])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\79526\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
import string
punkt_remove = string.punctuation #строка со всеми пунктуац. символами
def remove_punkt(text):
    return text.translate(str.maketrans('','',punkt_remove)) #создаем таблицу преобразования, которая будет использована для удаления всех символов пунктуации текста

In [101]:
import nltk
nltk.download('wordnet')
X_train, X_test, y_train, y_test = train_test_split(df.drop(['tweet_id', 'sentiment'], axis = 1), df['sentiment'], test_size=0.2, random_state=42)
train_obrabotka = {'content':[], 'sentiment': []}
train_obrabotka = pd.DataFrame(train_obrabotka)
for i, row in X_train.iterrows():
    n_row = pd.DataFrame({'content': lematize_word(remove_stopwords(remove_punkt(row['content']))), 'sentiment': y_train[i]}, index = [i])
    train_obrabotka = pd.concat([train_obrabotka, n_row]).reset_index(drop=True)

test_obrabotka = {'content':[], 'sentiment': []}
test_obrabotka = pd.DataFrame(test_obrabotka)
for i, row in X_test.iterrows():
    n_row = pd.DataFrame({'content': lematize_word(remove_stopwords(remove_punkt(row['content']))), 'sentiment': y_test[i]}, index = [i])
    test_obrabotka = pd.concat([test_obrabotka, n_row]).reset_index(drop=True)

X_train = train_obrabotka.drop(['sentiment'], axis=1)
y_train = train_obrabotka['sentiment']
X_test = test_obrabotka.drop(['sentiment'], axis=1)
y_test = test_obrabotka['sentiment']
X_train

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\79526\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,content
0,Thundershowers plus baseball equal awwww
1,The Hangover That movie gonna hilarious wish c...
2,Playing game I leave work Work til least 11PM ...
3,About stick work till freakin 2 Madd suck I wo...
4,SarahxATL u mean jack barakats wow u ever gone...
...,...
31995,Cleaning
31996,ooo goddddd vodafone trouble network
31997,watching run fat boy runhaha soo funny
31998,calvinharris make pork product theyve fucking ...


Bag of Words

In [102]:
## Объединение обучающего и тестового DataFrame
combined_data = pd.concat([X_train, X_test])

vectorizer = CountVectorizer() #экземпляр

#обучаем модель
vectorizer.fit(combined_data.content)

CountVectorizer()

In [103]:
X_train = vectorizer.transform(X_train.content)
X_test = vectorizer.transform(X_test.content)
y_train = y_train.values
y_test = y_test.values

Классификатор и расчёт метрик

In [104]:
classifier = LogisticRegression() #лог. регрессия
#classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)

y_pred = classifier.predict(X_test)

pres = precision_score(y_test,y_pred,average='weighted')
print(f"Precision: {pres}")

recall = recall_score(y_test,y_pred,average='weighted')
print(f"Recall: {recall}")

f1 = f1_score(y_test,y_pred,average='weighted')
print(f"F1:: {f1}")

Precision: 0.30513281728713376
Recall: 0.334875
F1:: 0.3085407028990827


C:\Users\79526\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\79526\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r